#1)GENERATING TXT FILE FROM VOTT_EXPORT_CSV

In [1]:
from PIL import Image
from os import path, makedirs
import os
import re
import pandas as pd
import sys
import argparse

In [2]:
#The below is the path of VOTT-CSV-EXPORT which got genrated from VOTT tool
Vott_folder_path="C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\Data\\Source_Images\\Training_Images\\vott-csv-export"

In [3]:
#Here we are defining path for "data_train.txt"
#Note we are just defining the path it can be any path of our choice
#Note we are just defining the path ,till now we havent created "data_train.txt"
#"data_train.txt" will contain the info needed for training purpose
#The info required for the training purpose will be extracted from CSV we have at "Vott_folder_path"
path1="C:\\Users\\lenovo\\DATA_TRAINED\\trying_yolo_master"
YOLO_filename = os.path.join(path1, "data_train.txt")

In [4]:
#We are defining path to access "data_classes.txt"
#This will contain the label name or the classes name for which we will be training our model
classes_filename = os.path.join(path1, "data_classes.txt")

In [5]:
#"Vott_csv" contain path to accsess "Annotations-export.csv" which is found inside "Vott_folder_path" 
Vott_csv = os.path.join(Vott_folder_path, "Annotations-export.csv")

In [6]:
# Readig csv from "Vott_csv" path defined above into  the dataframe "multi_df"
multi_df = pd.read_csv(Vott_csv)

In [7]:
#Displaying first 5 rows of the dataframe
multi_df.head()

,image,xmin,ymin,xmax,ymax,label
0,2Q__%20(10).jpg,1.051829,1.098039,222.759146,205.699346,fidget_spinner
1,2Q__%20(2).jpg,11.580882,9.926471,213.786765,202.573529,fidget_spinner
2,2Q__%20(3).jpg,0.000000,0.000000,225.000000,225.000000,fidget_spinner
3,2Q__%20(4).jpg,25.861667,2.797386,225.849815,190.571895,fidget_spinner
4,2Q__%20(5).jpg,20.772059,26.470588,201.102941,203.676471,fidget_spinner


In [8]:
#We are storing the distict labels from "label"
labels = multi_df["label"].unique()

In [9]:
#As we have onlly one unique label i.e "fidget_spinner",so below output contains only "fidget_spinner"
labels

array(['fidget_spinner'], dtype=object)

In [10]:
#Forming the dictionary "labeldict"
#uses "labels" array we got above
labeldict = dict(zip(labels, range(len(labels))))

In [11]:
labeldict

{'fidget_spinner': 0}

In [12]:
#dropping duplicates if any in the dataframe
multi_df.drop_duplicates(subset=None, keep="first", inplace=True)

In [13]:
#Generating text file from the dataframe i.e "multi_df" in our case
"""
Arguments
1)Dataframe which we got from csv in our case it is "multi_df"
2)Dictionary containing labels as keys and values as integers,in our case "labeldict"
3)Path which contains all the images for training the model in our case it is "Vott_folder_path"
4)This is the path where our text file will be generated ,in our case it is "YOLO_filename"
5)Here we have set "abs_path=False" as our "multi_df" does not contain full image path rather it contains only image names

"""
def convert_vott_csv_to_yolo(vott_df,labeldict,path,target_name,abs_path=False):
    # Encode labels according to labeldict if code's don't exist
    if not "code" in vott_df.columns:
        vott_df["code"] = vott_df["label"].apply(lambda x: labeldict[x])
    # Round float to ints
    for col in vott_df[["xmin", "ymin", "xmax", "ymax"]]:
        vott_df[col] = (vott_df[col]).apply(lambda x: round(x))

    # Create Yolo Text file
    last_image = ""
    txt_file = ""

    for index, row in vott_df.iterrows():
        if not last_image == row["image"]:
            if abs_path:
                txt_file += "\n" + row["image_path"] + " "
            else:
                txt_file += "\n" + os.path.join(path, row["image"]) + " "
            txt_file += ",".join(
                [
                    str(x)
                    for x in (row[["xmin", "ymin", "xmax", "ymax", "code"]].tolist())
                ]
            )
        else:
            txt_file += " "
            txt_file += ",".join(
                [
                    str(x)
                    for x in (row[["xmin", "ymin", "xmax", "ymax", "code"]].tolist())
                ]
            )
        last_image = row["image"]
    file = open(target_name, "w")
    file.write(txt_file[1:])
    file.close()
    return True

In [14]:
#caling above function to generate text file
convert_vott_csv_to_yolo(multi_df,labeldict,Vott_folder_path,YOLO_filename)

True

In [15]:
# Make classes file
#"classes_filename" we defined above and this path will contains all the label names on which training will be done
file = open(classes_filename, "w")

# Sort Dict by Values
SortedLabelDict = sorted(labeldict.items(), key=lambda x: x[1])
for elem in SortedLabelDict:
    file.write(elem[0] + "\n")
file.close()

#2)Training the model using info from above text file

In [16]:
import os
import sys
import argparse

In [17]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import (
    TensorBoard,
    ModelCheckpoint,
    ReduceLROnPlateau,
    EarlyStopping,
)

Using TensorFlow backend.


In [18]:
from PIL import Image
from time import time
import pickle
#yolo_helper conains some functions which we need to be used during our training process
#So below we import functions from "yolo_helper"
#Note we have created "Training_own_yolo" directory in site-packages and inside this we placed "yolo_helper.py" file
from Training_own_yolo.yolo_helper import get_random_data
from Training_own_yolo.yolo_helper import preprocess_true_boxes,yolo_body,tiny_yolo_body,yolo_loss

In [19]:
from Training_own_yolo.yolo_helper import (
    get_classes,
    get_anchors,
    create_model,
    create_tiny_model,
    data_generator,
    data_generator_wrapper
)

In [20]:
#"log_dir"-->This is the directory where our final weights obtained after training will be stored
log_dir="C:\\Users\\lenovo\\DATA_TRAINED\\trying_yolo_master"
#"anchors_path"-->Contains the predefined anchors for the YOLO
anchors_path="C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\2_Training\\src\\keras_yolo3\\model_data\\yolo_anchors.txt"
#It contains the weights path for yolo model named as "yolo.h5"
weights_path="C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\2_Training\\src\\keras_yolo3\\yolo.h5"

In [21]:
class_names = get_classes(classes_filename) #get_classes reads all the classes from text file into list
num_classes = len(class_names) #Getting total number of classses
anchors = get_anchors(anchors_path) #Getting anchors from the "yolo_anchors.txt" file in form numy array

In [22]:
#Since we are training the model only for one class so as we can see class_names consist of only"fidget_spinner" 
class_names

['fidget_spinner']

In [23]:
#After fetching the anchors from "anchors_path" we get numpy array of shape (9,2)
anchors

array([[ 10.,  13.],
       [ 16.,  30.],
       [ 33.,  23.],
       [ 30.,  61.],
       [ 62.,  45.],
       [ 59., 119.],
       [116.,  90.],
       [156., 198.],
       [373., 326.]])

In [24]:
#Here we are defining the input shape of the image which we will feed to our network
input_shape = (416, 416)  # multiple of 32, height, width

In [25]:
#Defining epochs to be used during the training of our model
epoch1, epoch2 = 51,51

In [26]:
#Yolo_tiny model uses different predefined anchors which when used len(anchors) will be 6
#Here our intention is to use yolo_model and not the "yolo_tiny"
#So we are chceking whether len(anchors)==6 if yes then we should create the yolo_tiny model as we are using "yolo-tiny_anchors.txt"
#"yolo-tiny_anchors" we can find in the same path as "yolo_anchors"
is_tiny_version = len(anchors) == 6  # default setting

In [27]:
#Since we are using the "yolo_anchors" so below output is justified
is_tiny_version

False

In [28]:
#Since is_tiny_version=False so here "else" block will get executed
#Note create_model we have imported from "yolo_helper" in above steps
if is_tiny_version:
    model = create_tiny_model(
        input_shape, anchors, num_classes, freeze_body=2, weights_path=weights_path
    )
else:
    model = create_model(
        input_shape, anchors, num_classes, freeze_body=2, weights_path=weights_path
    ) 

W0213 23:21:25.513850 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0213 23:21:25.516847 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0213 23:21:25.551827 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0213 23:21:25.552827 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0213 23:21:25.563821 19372 module_wrapper.py:139] From c:\users\len

Create YOLOv3 model with 9 anchors and 1 classes.


c:\users\lenovo\env\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 18) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
c:\users\lenovo\env\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
c:\users\lenovo\env\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 18) vs (255, 512, 1, 1)).
  weight_values[i].shape))
c:\users\lenovo\env\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
c:\users\lenovo\env\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_75 due to mismatch in shape ((1, 1, 2

Load weights C:\Users\lenovo\DATA_TRAINED\TrainYourOwnYOLO-master\TrainYourOwnYOLO-master\2_Training\src\keras_yolo3\yolo.h5.
Freeze the first 249 layers of total 252 layers.


In [29]:
#This block will basically take care of saving the model at various checkpoints at "log_dir" path defined above
#These are call back functions which will be used during our training
log_dir_time = os.path.join(log_dir, "{}".format(int(time())))
logging = TensorBoard(log_dir=log_dir_time)
checkpoint = ModelCheckpoint(
    os.path.join(log_dir, "checkpoint.h5"),
    monitor="val_loss",
    save_weights_only=True,
    save_best_only=True,
    period=5,
)

In [30]:
#These our call back funtions which will be used during our traiing
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10, verbose=1
)

In [31]:
#Defning the validation_split for the model during training
val_split=0.1

In [32]:
#YOLO_filename is the path to access "data_train.txt" which we generated from our "CSV" file initially
#"data_train.txt" file conatins full path of the images and bounding boxes corresponding to it and the labels in form of integers
#Here we will be storinng each line of the "data_train.txt" files as list into variable "lines"
with open(YOLO_filename) as f:
    lines = f.readlines()

In [33]:
#Lets see the contents of "lines" to get an idea
lines[:5]

['C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\Data\\Source_Images\\Training_Images\\vott-csv-export\\2Q__%20(10).jpg 1,1,223,206,0\n',
 'C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\Data\\Source_Images\\Training_Images\\vott-csv-export\\2Q__%20(2).jpg 12,10,214,203,0\n',
 'C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\Data\\Source_Images\\Training_Images\\vott-csv-export\\2Q__%20(3).jpg 0,0,225,225,0\n',
 'C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\Data\\Source_Images\\Training_Images\\vott-csv-export\\2Q__%20(4).jpg 26,3,226,191,0\n',
 'C:\\Users\\lenovo\\DATA_TRAINED\\TrainYourOwnYOLO-master\\TrainYourOwnYOLO-master\\Data\\Source_Images\\Training_Images\\vott-csv-export\\2Q__%20(5).jpg 21,26,201,204,0\n']

In [34]:
#Replacing the "\\" by "/" in "lines" variable containg "data_train.txt" content
#We will traverse through entire list content one by one and replace "\\" by "/"
# each item will be appended to "new_lines_list" after replacement
new_lines_list=list()
for item in lines:
    tmp=item.replace("\\", "/")
    new_lines_list.append(tmp)

In [35]:
lines=new_lines_list #assigning contents of "new_lines_list" to "lines" variable
np.random.shuffle(lines) #shuffling of the content of list "lines" 
num_val = int(len(lines) * val_split) #amount of data which we will use for validation in "lines"
num_train = len(lines) - num_val #amount of data which will be used for training purpose in "lines"

In [36]:
#Displaying the contents of the list "lines" after replacement of "\\" by "/"
lines[:5]
#As we can see in list "lines" each item contains image path and then corresponding bounding box and label

['C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20-%202020-02-03T202433.264.jpg 20,30,206,195,0\n',
 'C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20-%202020-02-03T202433.781.jpg 108,20,250,165,0\n',
 'C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(8).jpg 3,10,217,207,0\n',
 'C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(44).jpg 9,10,217,210,0\n',
 'C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(1).jpg 25,48,119,122,0 151,48,243,119,0\n']

In [37]:
# Train with frozen layers first, to get a stable loss.
# Adjust num epochs to your dataset. This step is enough to obtain a decent model.
if True:
    model.compile(
        optimizer=Adam(lr=1e-3),
        loss={
            # use custom yolo_loss Lambda layer.
            "yolo_loss": lambda y_true, y_pred: y_pred
        },
    )

    batch_size = 5
    print(
        "Train on {} samples, val on {} samples, with batch size {}.".format(
            num_train, num_val, batch_size
        )
    )
    history = model.fit_generator(
        data_generator_wrapper(
            lines[:num_train], batch_size, input_shape, anchors, num_classes
        ),
        steps_per_epoch=max(1, num_train // batch_size),
        validation_data=data_generator_wrapper(
            lines[num_train:], batch_size, input_shape, anchors, num_classes
        ),
        validation_steps=max(1, num_val // batch_size),
        epochs=epoch1,
        initial_epoch=0,
        callbacks=[logging, checkpoint],
    )
    model.save_weights(os.path.join(log_dir, "trained_weights_stage_1.h5"))

    step1_train_loss = history.history["loss"]

    file = open(os.path.join(log_dir_time, "step1_loss.npy"), "w")
    with open(os.path.join(log_dir_time, "step1_loss.npy"), "w") as f:
        for item in step1_train_loss:
            f.write("%s\n" % item)
    file.close()

    step1_val_loss = np.array(history.history["val_loss"])

    file = open(os.path.join(log_dir_time, "step1_val_loss.npy"), "w")
    with open(os.path.join(log_dir_time, "step1_val_loss.npy"), "w") as f:
        for item in step1_val_loss:
            f.write("%s\n" % item)
    file.close()

# Unfreeze and continue training, to fine-tune.
# Train longer if the result is unsatisfactory.
if True:
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(
        optimizer=Adam(lr=1e-4), loss={"yolo_loss": lambda y_true, y_pred: y_pred}
    )  # recompile to apply the change
    print("Unfreeze all layers.")

    batch_size = (
        4  # note that more GPU memory is required after unfreezing the body
    )
    print(
        "Train on {} samples, val on {} samples, with batch size {}.".format(
            num_train, num_val, batch_size
        )
    )
    history = model.fit_generator(
        data_generator_wrapper(
            lines[:num_train], batch_size, input_shape, anchors, num_classes
        ),
        steps_per_epoch=max(1, num_train // batch_size),
        validation_data=data_generator_wrapper(
            lines[num_train:], batch_size, input_shape, anchors, num_classes
        ),
        validation_steps=max(1, num_val // batch_size),
        epochs=epoch1 + epoch2,
        initial_epoch=epoch1,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping],
    )
    model.save_weights(os.path.join(log_dir, "trained_weights_final.h5"))
    step2_train_loss = history.history["loss"]

    file = open(os.path.join(log_dir_time, "step2_loss.npy"), "w")
    with open(os.path.join(log_dir_time, "step2_loss.npy"), "w") as f:
        for item in step2_train_loss:
            f.write("%s\n" % item)
    file.close()

    step2_val_loss = np.array(history.history["val_loss"])

    file = open(os.path.join(log_dir_time, "step2_val_loss.npy"), "w")
    with open(os.path.join(log_dir_time, "step2_val_loss.npy"), "w") as f:
        for item in step2_val_loss:
            f.write("%s\n" % item)
    file.close()

W0213 23:22:40.347550 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Train on 119 samples, val on 13 samples, with batch size 5.


W0213 23:22:41.463911 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0213 23:22:41.661798 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0213 23:22:43.126955 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0213 23:22:43.128954 19372 module_wrapper.py:139] From c:\users\lenovo\env\lib\site-packages\keras\callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/51
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/9k_%20(6).jpg
 1
42,101,226,185,0 125,3,222,92,0
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(70).jpg

 5
1,11,160,111,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/9k_%20(6).jpg', ' 142,101,226,185,0 125,3,222,92,0\n')('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(70).jpg', ' 51,11,160,111,0\n')

C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20-%202020-02-03T202444.428.jpg
 1
8,13,210,212,0
('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Traini

C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/2Q__%20(2).jpg
 1
2,10,214,203,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/2Q__%20(2).jpg', ' 12,10,214,203,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(36).jpg
 1
,1,233,215,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(36).jpg', ' 1,1,233,215,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(22).jpg
 1
6,17,244,160,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(22).jpg', ' 16,17,244,160,0\n')

C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(38).jpg
 2
0,22,208,197,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(38).jpg', ' 20,22,208,197,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(12).jpg
 2
4,31,187,180,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(12).jpg', ' 24,31,187,180,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(2).jpg
 2
,1,271,179,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(2).jpg', ' 2,1,271,179,0

C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/9k_%20(6).jpg
 1
42,101,226,185,0 125,3,222,92,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/9k_%20(6).jpg', ' 142,101,226,185,0 125,3,222,92,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(73).jpg
 4
7,62,176,164,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(73).jpg', ' 47,62,176,164,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/2Q__%20(10).jpg
 1
,1,223,206,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/2Q__%20(1

C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(78).jpg
 1
0,34,213,189,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(78).jpg', ' 10,34,213,189,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(96).jpg
 7
,49,133,186,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(96).jpg', ' 7,49,133,186,0\n')
C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(6).jpg
 2
9,57,175,188,0

('C:/Users/lenovo/DATA_TRAINED/TrainYourOwnYOLO-master/TrainYourOwnYOLO-master/Data/Source_Images/Training_Images/vott-csv-export/images%20(6).jpg', ' 29,57,175,188

KeyboardInterrupt: 